### Find Word Images And Copy

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

file_ext = 1000

# image result parameter
select_image_num = 1  # 10 image sample number big and equal this number
take_image_sample_num = 1000  # 50

In [3]:
word_lemma_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-2-Word In Visual Genome Merge"

word_lemma_data_path2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-3-Find Word Images Mask Fade"

path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-4-Find Word Images And Copy"

#Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [6]:
def take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50):
    '''take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then sort_target_column is sorting according to sort_ascending condition.\n
    Finally, taking sample_num each word_list values.\n 
    ex.\n
    take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat, word_list, "word", "search_text", sort_ascending=True, sample_num=50)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        df_select = df_source[df_source[f"{word_source_column}"] == word]
        df_select = df_select.sort_values(f"{sort_target_column}",key=lambda x:x.str.len(), ascending=sort_ascending).head(sample_num)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [7]:
#def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder):
#    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder)\n
#    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
#    output_path_folder from image_folder_path.\n
#    ex.\n
#    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
#    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
#    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", image_path, output_path_folder)
#    '''
#    df_search_result = pd.DataFrame()
#    for word in word_list:
#        path = f"{output_path_folder}/{word}"
#        Path(path).mkdir(parents=True, exist_ok=True)        
#        df_select = df_source[df_source[f"{word_source_column}"] == word]
#        for image_id in df_select[f"{image_id_column}"]:
#            image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
#            for l in image_file:
#                source = l # source directory
#                destination = path
#                shutil.copy2(source, destination)

In [8]:
def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, opt_column, image_folder_path, mask_folder_path, output_path_folder):
    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, mask_folder_path, output_path_folder)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
    output_path_folder from mask_folder_path (search in firstly) and image_folder_path. opt_column used for additional features\n
    ex.\n
    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path output_path_folder)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        try:
            path = f"{output_path_folder}/{word}"
            Path(path).mkdir(parents=True, exist_ok=True)        
            df_var = df_source[df_source[f"{word_source_column}"] == word]
            df_select = df_var.drop_duplicates(subset=[f"{image_id_column}"])  # ????
            imageid_list = list(df_select[f"{image_id_column}"])
            opt_column_list = list(df_select[f"{opt_column}"])
            data_list_zip = zip(imageid_list, opt_column_list)
            for image_id, opt in data_list_zip:
                opt_num = int(opt)

                #if isfile(f"{image_folder_path}/VG_Mask_Blur/{image_id}_{opt_num}_mask_blur.jpg"):  # ?????                
                #    image_file = glob.glob(f"{image_folder_path}/*/{image_id}_{opt_num}_mask_blur.jpg")

                if isfile(f"{mask_folder_path}/{image_id}_{opt_num}.jpg"):  # ?????                
                    image_file = glob.glob(f"{mask_folder_path}/{image_id}_{opt_num}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
                else:
                    image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
                    for l in image_file:
                        source = l # source directory
                        destination = path
                        shutil.copy2(source, destination)
        except:
            pass

In [9]:
image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"
mask_image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur"

In [10]:
image_folder_list = glob.glob(f"{image_path}/*")
image_folder_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K_2',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur']

#### Visual Genome Word Lemma Translate Data

In [11]:
#Pos_Tag = "VERB" # NOUN, VERB, ADJ, ADV, NUM, PRON, CCONJ, ADP, AUX

In [12]:
df_genome_word_lemma_concat = pd.read_csv(f"{word_lemma_data_path}/Visual_Genome_{file_ext}_Word_Lemma_Search_Result.csv")
df_genome_word_lemma_concat

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994,2920126305
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,129190150113
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,123317182233
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861,4615142168
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866,54188097
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


##### Noun

In [13]:
df_genome_word_lemma_concat_noun = df_genome_word_lemma_concat[df_genome_word_lemma_concat["POS1"] == "NOUN"]
df_genome_word_lemma_concat_noun

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
844,NOUN,NaN,şu,şu,şu,this,this,1377715,this,525,3323001266
845,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184
846,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,622869100
847,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,547564258
848,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,77120262298
...,...,...,...,...,...,...,...,...,...,...,...
192518,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,desk lamp,2354353,64862790
192519,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,white desk,2389995,2545371147
192520,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,glass desk,2389540,4343170199
192521,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,small desk,2348802,180101241147


In [14]:
df_genome_word_lemma_concat_noun_select = df_genome_word_lemma_concat_noun.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_noun_select

,word,image_id
0,acele,True
1,adam,True
2,adama,True
3,adamlar,True
4,adamı,True
...,...,...
411,şeyin,True
412,şeyler,True
413,şeyleri,True
414,şimdiye,True


In [15]:
noun_word_select_list = df_genome_word_lemma_concat_noun_select[df_genome_word_lemma_concat_noun_select["image_id"] == True]["word"].to_list()
#noun_word_select_list

In [16]:
df_genome_word_lemma_concat_noun_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(noun_word_select_list)]
df_genome_word_lemma_concat_noun_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
844,NOUN,NaN,şu,şu,şu,this,this,1377715,this,525,3323001266
845,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,127159144184
846,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,622869100
847,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,547564258
848,NOUN,NaN,şu,şu,şu,this,this,1377715,this,2350323,77120262298
...,...,...,...,...,...,...,...,...,...,...,...
192518,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,desk lamp,2354353,64862790
192519,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,white desk,2389995,2545371147
192520,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,glass desk,2389540,4343170199
192521,NOUN,NaN,sırada,sıra,sıra,in line,desk,69586,small desk,2348802,180101241147


In [17]:
noun_word_list = list(set(df_genome_word_lemma_concat_noun_select_result["word"]))
noun_lemma_list = list(set(df_genome_word_lemma_concat_noun_select_result["lemma.spacy"]))

In [18]:
df_genome_noun_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_noun_select_result, noun_word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_noun_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NOUN,NaN,çocuklar,çocuk,çocuk,children,child,341288,child,2348313,8530451118
1,NOUN,NaN,çocuklar,çocuk,çocuk,children,child,341288,child,2350717,128110101167
2,NOUN,NaN,çocuklar,çocuk,çocuk,children,child,341288,child,2350220,1991947160
3,NOUN,NaN,çocuklar,çocuk,çocuk,children,child,341288,child,2350211,3612061396
4,NOUN,NaN,çocuklar,çocuk,çocuk,children,child,341288,child,2350211,31819316951
...,...,...,...,...,...,...,...,...,...,...,...
84987,NOUN,NaN,oyun,oyun,oyun,game,game,141855,what game is this,2366019,0
84988,NOUN,NaN,oyun,oyun,oyun,game,game,141855,what game is this,2368584,0
84989,NOUN,NaN,oyun,oyun,oyun,game,game,141855,what type of game,2370504,0
84990,NOUN,NaN,oyun,oyun,oyun,game,game,141855,what game is this,2407596,0


In [19]:
df_genome_noun_sample_result["lemma.spacy"].nunique()

276

In [20]:
noun_output_path_folder = f"/home/kurubal/Downloads/Image_Select/NOUN"

Path(noun_output_path_folder).mkdir(parents=True, exist_ok=True)

In [21]:
# create folder
for lemma in noun_lemma_list:
    path = f"{noun_output_path_folder}/ALL_NOUN_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_noun_sample_result[df_genome_noun_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [22]:
# for lemma
create_word_folder_and_copy_image(df_genome_noun_sample_result, noun_lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path, noun_output_path_folder)

##### Verb

In [23]:
df_genome_word_lemma_concat_verb = df_genome_word_lemma_concat[df_genome_word_lemma_concat["POS1"] == "VERB"]
df_genome_word_lemma_concat_verb

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
2320,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2321637,1308718190
2321,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2412863,2404701598
2322,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what this is,2388466,34576516
2323,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what is not allowed,2399091,553912620
2324,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,"what , white",2408062,365513498
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [24]:
df_genome_word_lemma_concat_verb_select = df_genome_word_lemma_concat_verb.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_verb_select

,word,image_id
0,al,True
1,alabilir,True
2,alacağım,True
3,aldı,True
4,aldım,True
...,...,...
274,çık,True
275,çıktı,True
276,öldü,True
277,öldürmek,True


In [25]:
verb_word_select_list = df_genome_word_lemma_concat_verb_select[df_genome_word_lemma_concat_verb_select["image_id"] == True]["word"].to_list()
#verb_word_select_list

In [26]:
df_genome_word_lemma_concat_verb_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(verb_word_select_list)]
df_genome_word_lemma_concat_verb_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
2320,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2321637,1308718190
2321,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what,2412863,2404701598
2322,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what this is,2388466,34576516
2323,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,what is not allowed,2399091,553912620
2324,VERB,NaN,gerektiğini,gerek,gerek,what,necessary,120666,"what , white",2408062,365513498
...,...,...,...,...,...,...,...,...,...,...,...
192717,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616,276026737
192718,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604,376517459
192719,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,95132301236
192720,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,2211405194


In [27]:
verb_word_list = list(set(df_genome_word_lemma_concat_verb_select_result["word"]))
verb_lemma_list = list(set(df_genome_word_lemma_concat_verb_select_result["lemma.spacy"]))

In [28]:
df_genome_verb_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_verb_select_result, verb_word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_verb_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,VERB,NaN,oldu,ol,ol,happened,be,1141161,be,2320303,133179254183
1,VERB,NaN,oldu,ol,ol,happened,be,1141161,be,2321043,2439143123
2,VERB,NaN,oldu,ol,ol,happened,be,1141161,be quiet,2384470,0
3,VERB,NaN,oldu,ol,ol,happened,be,1141161,be light,2405336,0
4,VERB,NaN,oldu,ol,ol,happened,be,1141161,to be ill,2369214,0
...,...,...,...,...,...,...,...,...,...,...,...
50781,VERB,NEG,unutma,unut,unut,don't forget,forget,85318,hamilton beach set it and forget it,713876,286274464700
50782,VERB,NEG,unutma,unut,unut,don't forget,forget,85318,the diner s left thumb sports a don t forget band,2343867,77614041
50783,VERB,NEG,unutma,unut,unut,don't forget,forget,85318,don t forget to flush written on the wall in t...,2364432,7193115125
50784,VERB,NEG,unutma,unut,unut,don't forget,forget,85318,wall image in mirror reading in reverse don t ...,2364432,7191122123


In [29]:
df_genome_verb_sample_result["lemma.spacy"].nunique()

73

In [30]:
verb_output_path_folder = f"/home/kurubal/Downloads/Image_Select/VERB"

Path(verb_output_path_folder).mkdir(parents=True, exist_ok=True)

In [31]:
# create folder
for lemma in verb_lemma_list:
    path = f"{verb_output_path_folder}/ALL_VERB_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_verb_sample_result[df_genome_verb_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [32]:
# for lemma
create_word_folder_and_copy_image(df_genome_verb_sample_result, verb_lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path, verb_output_path_folder)

##### Adjective Adverb Pronouns

In [33]:
df_genome_word_lemma_concat_adj_adv_pron = df_genome_word_lemma_concat[(df_genome_word_lemma_concat["POS1"] == "ADJ") | (df_genome_word_lemma_concat["POS1"] == "ADV") | (df_genome_word_lemma_concat["POS1"] == "PRON")]
df_genome_word_lemma_concat_adj_adv_pron

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
284,PRON,NaN,bu,bu,bu,this,this,11062659,this,525,3323001266
285,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,127159144184
286,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,622869100
287,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,547564258
288,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,77120262298
...,...,...,...,...,...,...,...,...,...,...,...
191222,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,the james bond logo on the train,2412980,2849138225
191223,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,advertisement for a james bond movie,2375682,2124673267
191224,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,blue words mudie bond written on bike,2324286,7724160309
191225,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,little bond boy with food in his mouth,2384940,49723100


In [34]:
df_genome_word_lemma_concat_adj_adv_pron_select = df_genome_word_lemma_concat_adj_adv_pron.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_adj_adv_pron_select

,word,image_id
0,acaba,True
1,acı,True
2,altına,True
3,arada,True
4,arasında,True
...,...,...
230,üstünde,True
231,şimdi,True
232,şuna,True
233,şunu,True


In [35]:
adj_adv_pron_word_select_list = df_genome_word_lemma_concat_adj_adv_pron_select[df_genome_word_lemma_concat_adj_adv_pron_select["image_id"] == True]["word"].to_list()
#adj_adv_pron_word_select_list

In [36]:
df_genome_word_lemma_concat_adj_adv_pron_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(adj_adv_pron_word_select_list)]
df_genome_word_lemma_concat_adj_adv_pron_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
284,PRON,NaN,bu,bu,bu,this,this,11062659,this,525,3323001266
285,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,127159144184
286,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,622869100
287,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,547564258
288,PRON,NaN,bu,bu,bu,this,this,11062659,this,2350323,77120262298
...,...,...,...,...,...,...,...,...,...,...,...
191222,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,the james bond logo on the train,2412980,2849138225
191223,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,advertisement for a james bond movie,2375682,2124673267
191224,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,blue words mudie bond written on bike,2324286,7724160309
191225,ADJ,NaN,bağlı,bağ,bağ,connected,bond,83182,little bond boy with food in his mouth,2384940,49723100


In [37]:
adj_adv_pron_word_list = list(set(df_genome_word_lemma_concat_adj_adv_pron_select_result["word"]))
adj_adv_pron_lemma_list = list(set(df_genome_word_lemma_concat_adj_adv_pron_select_result["lemma.spacy"]))

In [38]:
df_genome_adj_adv_pron_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_adj_adv_pron_select_result, adj_adv_pron_word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_adj_adv_pron_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,PRON,NaN,siz,siz,siz,you,you,591994,you,2345948,234932355
1,PRON,NaN,siz,siz,siz,you,you,591994,you,488,9096640406
2,PRON,NaN,siz,siz,siz,you,you,591994,you,488,8858700411
3,PRON,NaN,siz,siz,siz,you,you,591994,you,488,10690339479
4,PRON,NaN,siz,siz,siz,you,you,591994,you,488,109112236481
...,...,...,...,...,...,...,...,...,...,...,...
48072,ADJ,NaN,gerçek,gerçek,gerçek,real,real,414956,what would happen if real bricks were loaded,2364632,0
48073,ADJ,NaN,gerçek,gerçek,gerçek,real,real,414956,how many animals are photographed real and fake,2409702,0
48074,ADJ,NaN,gerçek,gerçek,gerçek,real,real,414956,blue and white on the front and pink on the real,2410901,0
48075,ADJ,NaN,gerçek,gerçek,gerçek,real,real,414956,is a real estate listing to show in public people,2365810,0


In [39]:
df_genome_adj_adv_pron_sample_result["lemma.spacy"].nunique()

179

In [40]:
adj_adv_pron_output_path_folder = f"/home/kurubal/Downloads/Image_Select/ADJ_ADV_PRON"

Path(adj_adv_pron_output_path_folder).mkdir(parents=True, exist_ok=True)

In [41]:
# create folder
for lemma in adj_adv_pron_lemma_list:
    path = f"{adj_adv_pron_output_path_folder}/ALL_ADJ_ADV_PRON_FOLDER/{lemma}"
    Path(path).mkdir(parents=True, exist_ok=True)
    df_var = df_genome_adj_adv_pron_sample_result[df_genome_adj_adv_pron_sample_result["lemma.spacy"] == f"{lemma}"]
    for word in set(df_var["word"]):
        path2 = f"{path}/{word}"
        Path(path2).mkdir(parents=True, exist_ok=True)

In [42]:
# for lemma
create_word_folder_and_copy_image(df_genome_adj_adv_pron_sample_result, adj_adv_pron_lemma_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path, adj_adv_pron_output_path_folder)

##### Other

In [43]:
df_genome_word_lemma_concat_other = df_genome_word_lemma_concat[~((df_genome_word_lemma_concat["POS1"] == "NOUN") | (df_genome_word_lemma_concat["POS1"] == "VERB") | 
(df_genome_word_lemma_concat["POS1"] == "ADJ") | (df_genome_word_lemma_concat["POS1"] == "ADV") | (df_genome_word_lemma_concat["POS1"] == "PRON"))]
df_genome_word_lemma_concat_other

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994,2920126305
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,129190150113
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,123317182233
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861,4615142168
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866,54188097
...,...,...,...,...,...,...,...,...,...,...,...
165590,AUX,NaN,miyim,mi,mi,am i,mi,388689,the letters mi on the street sign,2386741,523121973
165591,AUX,NaN,miyim,mi,mi,am i,mi,388689,coburg rd mi min inside right arrow,2366858,30110170180
165592,AUX,NaN,miyim,mi,mi,am i,mi,388689,oakway shopping ctr mi min in right arrow,2366858,40162166213
165593,AUX,NaN,miyim,mi,mi,am i,mi,388689,valley river center mi min straight ahead,2366858,40164166247


In [44]:
df_genome_word_lemma_concat_other_select = df_genome_word_lemma_concat_other.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_other_select

,word,image_id
0,altı,True
1,ama,True
2,ancak,True
3,beş,True
4,bin,True
5,bir,True
6,boyunca,True
7,da,True
8,de,True
9,değil,True


In [45]:
other_word_select_list = df_genome_word_lemma_concat_other_select[df_genome_word_lemma_concat_other_select["image_id"] == True]["word"].to_list()
#other_word_select_list

In [46]:
df_genome_word_lemma_concat_other_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(other_word_select_list)]
df_genome_word_lemma_concat_other_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994,2920126305
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,129190150113
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,123317182233
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861,4615142168
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866,54188097
...,...,...,...,...,...,...,...,...,...,...,...
165590,AUX,NaN,miyim,mi,mi,am i,mi,388689,the letters mi on the street sign,2386741,523121973
165591,AUX,NaN,miyim,mi,mi,am i,mi,388689,coburg rd mi min inside right arrow,2366858,30110170180
165592,AUX,NaN,miyim,mi,mi,am i,mi,388689,oakway shopping ctr mi min in right arrow,2366858,40162166213
165593,AUX,NaN,miyim,mi,mi,am i,mi,388689,valley river center mi min straight ahead,2366858,40164166247


In [47]:
other_word_list = list(set(df_genome_word_lemma_concat_other_select_result["word"]))
other_lemma_list = list(set(df_genome_word_lemma_concat_other_select_result["lemma.spacy"]))

In [48]:
df_genome_other_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_other_select_result, other_word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_other_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,üçüncü,üçüncü,üç,third,third,69620,third,2343467,1726163177
1,NUM,NaN,üçüncü,üçüncü,üç,third,third,69620,third,2345685,191123299
2,NUM,NaN,üçüncü,üçüncü,üç,third,third,69620,third,2380678,6957283100
3,NUM,NaN,üçüncü,üçüncü,üç,third,third,69620,third,2344763,115184148215
4,NUM,NaN,üçüncü,üçüncü,üç,third,third,69620,third,2341030,12137106403
...,...,...,...,...,...,...,...,...,...,...,...
8862,NUM,NaN,bir,bir,bir,a,a,18835735,who has a mane,2406250,0
8863,NUM,NaN,bir,bir,bir,a,a,18835735,who has a fork,2333157,0
8864,NUM,NaN,bir,bir,bir,a,a,18835735,where is a fan,2394383,0
8865,NUM,NaN,bir,bir,bir,a,a,18835735,who has a tail,2376886,0


In [49]:
df_genome_other_sample_result["word"].nunique()

53

In [53]:
other_output_path_folder = f"/home/kurubal/Downloads/Image_Select/OTHER"

Path(other_output_path_folder).mkdir(parents=True, exist_ok=True)

In [54]:
# create folder
for word in other_word_list:
    path = f"{other_output_path_folder}/ALL_OTHER_FOLDER/{word}"
    Path(path).mkdir(parents=True, exist_ok=True)

In [52]:
# for lemma
create_word_folder_and_copy_image(df_genome_other_sample_result, other_word_list, "lemma.spacy", "image_id", "num", image_path, mask_image_path, other_output_path_folder)

In [ ]:
# create folder
# for word in word_list:
#    path = f"{output_path_folder}/ALL_{Pos_Tag}_FOLDER/{word}"
#    Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
# for word
#create_word_folder_and_copy_image(df_genome_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path ,output_path_folder)

In [ ]:
## Check Result 1
#df_genome_word_lemma_concat_coor = pd.read_csv(f"{word_lemma_data_path2}/Visual_Genome_{file_ext}_Word_Lemma_Coordinate_Search_Result.csv")
#df_genome_word_lemma_concat_coor = df_genome_word_lemma_concat_coor.drop(["height","width","x_koor","y_koor"], axis=1)
#df_genome_word_lemma_concat_coor

In [ ]:
# Check Result 2
#df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat_select_result,df_genome_word_lemma_concat_coor,how="left", on=["word","image_id","word_en_translate","lemma_en_translate","lemma.spacy","stem","num"])
##df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat, df_genome_word_lemma_concat_coor,how="left", on=["word","image_id"])
#df_genome_word_lemma_concat_select_result_coor_merge = df_genome_word_lemma_concat_select_result_coor_merge.drop_duplicates()
##df_genome_word_lemma_concat_select_result_coor_merge["num"] = df_genome_word_lemma_concat_select_result_coor_merge["num"].fillna(0)
#df_genome_word_lemma_concat_select_result_coor_merge

In [ ]:
#df_genome_word_lemma_concat_select_result_coor_merge["word"].nunique()

In [ ]:
#word_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["word"]))
#lemma_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["lemma.spacy"]))

In [ ]:
#df_genome_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_select_result_coor_merge, word_list, "word", 
#                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
#df_genome_sample_result

In [ ]:
#df_genome_sample_result["word"].nunique()